# В РАЗРАБОТКЕ

1. Проверить гипотезу: параметр `price` перчаток с параметром `case == Clutch Case`, в среднем сильнее зависит от показателя `wear_value`, чем с параметром `case == Glove Case, Operation Hydra Case`
2. Найти перчатки, у которых показатель `wear_value`: а) изменяется больше, чем у всех остальных; б) изменяется меньше, чем у всех остальных
3. Составить рейтинг перчаток, которые имеют сильную зависимость цены от `wear_value`, но которые можно чаще всего купить за стандартную цену. Посчитать среднюю прибыль перепродажи таких предметов за `n` дней.

In [52]:
import pandas as pd

In [53]:
df = pd.read_csv('data.csv')
df

,market_hash_name,price,wear_value,sold_at,quality,case
0,★ Moto Gloves | Cool Mint (Well-Worn),131.66,-1.000000,1513873445,WW,"Glove Case, Operation Hydra Case"
1,★ Driver Gloves | Lunar Weave (Battle-Scarred),73.74,0.585385,1561561294,BS,"Glove Case, Operation Hydra Case"
2,★ Hand Wraps | Leather (Battle-Scarred),94.50,0.747108,1538346436,BS,"Glove Case, Operation Hydra Case"
3,★ Specialist Gloves | Emerald Web (Minimal Wear),609.99,0.127194,1555932285,MW,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Imperial Plaid (Well-Worn),225.99,0.382708,1559791453,WW,Clutch Case
...,...,...,...,...,...,...
26236,★ Hand Wraps | Badlands (Well-Worn),135.00,0.382254,1587296787,WW,"Glove Case, Operation Hydra Case"
26237,★ Hand Wraps | Badlands (Battle-Scarred),68.19,0.762055,1553500961,BS,"Glove Case, Operation Hydra Case"
26238,★ Hand Wraps | Arboreal (Field-Tested),100.25,0.366468,1595437849,FT,Clutch Case
26239,★ Sport Gloves | Omega (Battle-Scarred),199.00,0.625256,1537803224,BS,Clutch Case


# Подготовка данных:

Уберем окончание значений `market_hash_name	`, которое указывает на `quality`:

In [54]:
for index, row in df.iterrows():
    df.loc[index, 'market_hash_name'] = row['market_hash_name'][: row['market_hash_name'].index('(')]
df

,market_hash_name,price,wear_value,sold_at,quality,case
0,★ Moto Gloves | Cool Mint,131.66,-1.000000,1513873445,WW,"Glove Case, Operation Hydra Case"
1,★ Driver Gloves | Lunar Weave,73.74,0.585385,1561561294,BS,"Glove Case, Operation Hydra Case"
2,★ Hand Wraps | Leather,94.50,0.747108,1538346436,BS,"Glove Case, Operation Hydra Case"
3,★ Specialist Gloves | Emerald Web,609.99,0.127194,1555932285,MW,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Imperial Plaid,225.99,0.382708,1559791453,WW,Clutch Case
...,...,...,...,...,...,...
26236,★ Hand Wraps | Badlands,135.00,0.382254,1587296787,WW,"Glove Case, Operation Hydra Case"
26237,★ Hand Wraps | Badlands,68.19,0.762055,1553500961,BS,"Glove Case, Operation Hydra Case"
26238,★ Hand Wraps | Arboreal,100.25,0.366468,1595437849,FT,Clutch Case
26239,★ Sport Gloves | Omega,199.00,0.625256,1537803224,BS,Clutch Case


Значение `price` должно быть строго больше нуля.

Исключим из выборки строки, которые не удовлетворяют данному условию, если они имеются:

In [55]:
df = df[df['price'] > 0]
df

,market_hash_name,price,wear_value,sold_at,quality,case
0,★ Moto Gloves | Cool Mint,131.66,-1.000000,1513873445,WW,"Glove Case, Operation Hydra Case"
1,★ Driver Gloves | Lunar Weave,73.74,0.585385,1561561294,BS,"Glove Case, Operation Hydra Case"
2,★ Hand Wraps | Leather,94.50,0.747108,1538346436,BS,"Glove Case, Operation Hydra Case"
3,★ Specialist Gloves | Emerald Web,609.99,0.127194,1555932285,MW,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Imperial Plaid,225.99,0.382708,1559791453,WW,Clutch Case
...,...,...,...,...,...,...
26236,★ Hand Wraps | Badlands,135.00,0.382254,1587296787,WW,"Glove Case, Operation Hydra Case"
26237,★ Hand Wraps | Badlands,68.19,0.762055,1553500961,BS,"Glove Case, Operation Hydra Case"
26238,★ Hand Wraps | Arboreal,100.25,0.366468,1595437849,FT,Clutch Case
26239,★ Sport Gloves | Omega,199.00,0.625256,1537803224,BS,Clutch Case


Значение `wear_value` должно лежать в интервале (0, 1)

Будем рассматривать только строки с корректным значением `wear_value`:

In [56]:
df = df[(df['wear_value'] > 0) & (df['wear_value'] < 1)]
df

,market_hash_name,price,wear_value,sold_at,quality,case
1,★ Driver Gloves | Lunar Weave,73.74,0.585385,1561561294,BS,"Glove Case, Operation Hydra Case"
2,★ Hand Wraps | Leather,94.50,0.747108,1538346436,BS,"Glove Case, Operation Hydra Case"
3,★ Specialist Gloves | Emerald Web,609.99,0.127194,1555932285,MW,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Imperial Plaid,225.99,0.382708,1559791453,WW,Clutch Case
5,★ Hand Wraps | Leather,105.77,0.650310,1536427996,BS,"Glove Case, Operation Hydra Case"
...,...,...,...,...,...,...
26236,★ Hand Wraps | Badlands,135.00,0.382254,1587296787,WW,"Glove Case, Operation Hydra Case"
26237,★ Hand Wraps | Badlands,68.19,0.762055,1553500961,BS,"Glove Case, Operation Hydra Case"
26238,★ Hand Wraps | Arboreal,100.25,0.366468,1595437849,FT,Clutch Case
26239,★ Sport Gloves | Omega,199.00,0.625256,1537803224,BS,Clutch Case


## Найти перчатки, у которых показатель wear_value: а) изменяется больше, чем у всех остальных; б) изменяется меньше, чем у всех остальных

Для решения данной задачи необходимо найти дисперсию

В данном случае дельта степеней свободы равна 0


In [57]:
import numpy as np

Получим список всех `market_hash_name` и список всех `quality`

In [58]:
market_hash_names = df['market_hash_name'].unique()
qualities = df['quality'].unique()

Для каждой пары `market_hash_name` и `quality` найдем значение дисперсии:

In [59]:
import itertools

In [60]:
vars_info = []
for market_hash_name, quality in itertools.product(market_hash_names, qualities):
    vars_info.append({'market_hash_name': market_hash_name,
                      'quality': quality, 
                      'var': df[(df['market_hash_name'] == market_hash_name) & (df['quality'] == quality)]['wear_value'].var()})
vars_info = pd.DataFrame(vars_info)
vars_info

,market_hash_name,quality,var
0,★ Driver Gloves | Lunar Weave,BS,0.011211
1,★ Driver Gloves | Lunar Weave,MW,0.000182
2,★ Driver Gloves | Lunar Weave,WW,0.000302
3,★ Driver Gloves | Lunar Weave,FT,0.005614
4,★ Driver Gloves | Lunar Weave,FN,0.000003
...,...,...,...
235,★ Sport Gloves | Amphibious,BS,0.010551
236,★ Sport Gloves | Amphibious,MW,0.000384
237,★ Sport Gloves | Amphibious,WW,0.000361
238,★ Sport Gloves | Amphibious,FT,0.005724


Найдем максимальные и минимальные значения дисперсии для каждого значения `quality`:

In [61]:
maxs = []
mins = []
for quality in qualities:
    max_var = vars_info[vars_info['quality'] == quality]['var'].max()
    min_var = vars_info[vars_info['quality'] == quality]['var'].min()
    maxs += vars_info[vars_info['var'] == max_var].to_dict(orient='records')
    mins += vars_info[vars_info['var'] == min_var].to_dict(orient='records')

Список перчаток каждого качества, у которых значение `wear_value` изменяется сильнее, чем у остальных:

In [62]:
pd.DataFrame(maxs)

,market_hash_name,quality,var
0,★ Sport Gloves | Vice,BS,0.012481
1,★ Hydra Gloves | Mangrove,MW,0.000557
2,★ Moto Gloves | Polygon,WW,0.000520
3,★ Hydra Gloves | Case Hardened,FT,0.006651
4,★ Sport Gloves | Vice,FN,0.000021


Список перчаток каждого качества, у которых значение `wear_value` изменяется слабее, чем у остальных:

In [63]:
pd.DataFrame(mins)

,market_hash_name,quality,var
0,★ Specialist Gloves | Mogul,BS,7.905624e-03
1,★ Driver Gloves | Racing Green,MW,1.715146e-04
2,★ Moto Gloves | Boom!,WW,1.611888e-04
3,★ Bloodhound Gloves | Bronzed,FT,4.228206e-03
4,★ Specialist Gloves | Crimson Kimono,FN,7.932144e-10


Получены значения дисперции для каждого вида перчаток на всей выборке. 
Проблема в том, что дисперсии считались для разного числа продаж. 

Посмотрим, сколько было совершено продаж перчаток каждого вида:

In [64]:
solds_count = []
for market_hash_name, quality in itertools.product(market_hash_names, qualities):
    solds_count.append({'market_hash_name': market_hash_name,
                      'quality': quality,
                      'total_solds': df[(df['market_hash_name'] == market_hash_name) & (df['quality'] == quality)]['wear_value'].count()})
solds_count = pd.DataFrame(solds_count)
solds_count.sort_values('total_solds')

,market_hash_name,quality,total_solds
119,★ Specialist Gloves | Buckshot,FN,0
234,★ Sport Gloves | Omega,FN,0
99,★ Hand Wraps | Cobalt Skulls,FN,0
54,★ Hydra Gloves | Mangrove,FN,0
24,★ Bloodhound Gloves | Snakebite,FN,0
...,...,...,...
208,★ Specialist Gloves | Forest DDPAT,FT,147
15,★ Driver Gloves | Imperial Plaid,BS,147
153,★ Driver Gloves | Diamondback,FT,147
78,★ Hand Wraps | Duct Tape,FT,147


Исключим из выборки элементы, у которых количество продаж равно `0` или `1`:

In [65]:
solds_count = solds_count[solds_count['total_solds'] > 1]
solds_count

,market_hash_name,quality,total_solds
0,★ Driver Gloves | Lunar Weave,BS,141
1,★ Driver Gloves | Lunar Weave,MW,145
2,★ Driver Gloves | Lunar Weave,WW,145
3,★ Driver Gloves | Lunar Weave,FT,135
4,★ Driver Gloves | Lunar Weave,FN,4
...,...,...,...
233,★ Sport Gloves | Omega,FT,139
235,★ Sport Gloves | Amphibious,BS,146
236,★ Sport Gloves | Amphibious,MW,89
237,★ Sport Gloves | Amphibious,WW,67


Найдем 1, 2, 3 квантили продаж для каждого качества:

In [66]:
quantiles = pd.DataFrame()

for quality in qualities:
    column_values = []
    for quantile_ in (0.25, 0.5, 0.75):
        column_values += list(solds_count[solds_count['quality'] == quality].quantile(quantile_).values)
    quantiles[quality] = column_values
quantiles.set_index(np.array(['25%', '50%', '75%']), inplace=True)

In [67]:
quantiles

,BS,MW,WW,FT,FN
25%,137.0,105.25,84.75,135.00,2.0
50%,141.0,134.00,117.00,139.00,3.0
75%,144.0,139.25,134.25,141.25,5.0


Получим значения дисперсий `wear_value` для каждого качества на каждом квантиле из уже найденных дисперсий:

In [68]:
vars_info_tmp = vars_info.copy()

In [69]:
vars_info_tmp = vars_info_tmp.merge(solds_count)


Первый квантиль:

In [70]:
maxs = []
mins = []
for quality in qualities:
    max_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (vars_info_tmp['total_solds'] <= quantiles[quality][0])]['var'].max()
    min_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (vars_info_tmp['total_solds'] <= quantiles[quality][0])]['var'].min()
    maxs += vars_info_tmp[vars_info_tmp['var'] == max_var].to_dict(orient='records')
    mins += vars_info_tmp[vars_info_tmp['var'] == min_var].to_dict(orient='records')

In [71]:
pd.DataFrame(maxs)

,market_hash_name,quality,var,total_solds
0,★ Hand Wraps | Arboreal,BS,0.011873,128
1,★ Moto Gloves | Spearmint,MW,0.000507,73
2,★ Hand Wraps | Arboreal,WW,0.000488,70
3,★ Driver Gloves | Crimson Weave,FT,0.006598,135
4,★ Sport Gloves | Vice,FN,0.000021,2


In [72]:
pd.DataFrame(mins)

,market_hash_name,quality,var,total_solds
0,★ Specialist Gloves | Mogul,BS,7.905624e-03,137
1,★ Sport Gloves | Hedge Maze,MW,1.729535e-04,85
2,★ Sport Gloves | Vice,WW,1.806081e-04,69
3,★ Driver Gloves | Imperial Plaid,FT,4.698191e-03,130
4,★ Specialist Gloves | Crimson Kimono,FN,7.932144e-10,2


Второй квантиль:

In [73]:
maxs = []
mins = []
for quality in qualities:
    max_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (quantiles[quality][0] <= vars_info_tmp['total_solds']) & (vars_info_tmp['total_solds'] >= quantiles[quality][1])]['var'].max()
    min_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (quantiles[quality][0] <= vars_info_tmp['total_solds']) & (vars_info_tmp['total_solds'] >= quantiles[quality][1])]['var'].min()
    maxs += vars_info_tmp[vars_info_tmp['var'] == max_var].to_dict(orient='records')
    mins += vars_info_tmp[vars_info_tmp['var'] == min_var].to_dict(orient='records')

In [74]:
pd.DataFrame(maxs)

,market_hash_name,quality,var,total_solds
0,★ Sport Gloves | Vice,BS,0.012481,141
1,★ Driver Gloves | Diamondback,MW,0.000479,135
2,★ Driver Gloves | Convoy,WW,0.000482,140
3,★ Hydra Gloves | Case Hardened,FT,0.006651,141
4,★ Sport Gloves | Arid,FN,0.000019,3


In [75]:
pd.DataFrame(mins)

,market_hash_name,quality,var,total_solds
0,★ Specialist Gloves | Emerald Web,BS,8.008868e-03,143
1,★ Driver Gloves | Racing Green,MW,1.715146e-04,145
2,★ Moto Gloves | Boom!,WW,1.611888e-04,135
3,★ Bloodhound Gloves | Bronzed,FT,4.228206e-03,146
4,★ Driver Gloves | Racing Green,FN,4.186478e-07,4


Третий квантиль:

In [76]:
maxs = []
mins = []
for quality in qualities:
    max_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (quantiles[quality][1] <= vars_info_tmp['total_solds']) & (vars_info_tmp['total_solds'] >= quantiles[quality][2])]['var'].max()
    min_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (quantiles[quality][1] <= vars_info_tmp['total_solds']) & (vars_info_tmp['total_solds'] >= quantiles[quality][2])]['var'].min()
    maxs += vars_info_tmp[vars_info_tmp['var'] == max_var].to_dict(orient='records')
    mins += vars_info_tmp[vars_info_tmp['var'] == min_var].to_dict(orient='records')

In [77]:
pd.DataFrame(maxs)

,market_hash_name,quality,var,total_solds
0,★ Hand Wraps | Cobalt Skulls,BS,0.011728,146
1,★ Bloodhound Gloves | Guerrilla,MW,0.000399,146
2,★ Driver Gloves | Convoy,WW,0.000482,140
3,★ Hand Wraps | Duct Tape,FT,0.006078,147
4,★ Bloodhound Gloves | Charred,FN,0.000013,5


In [78]:
pd.DataFrame(mins)

,market_hash_name,quality,var,total_solds
0,★ Specialist Gloves | Foundation,BS,0.008430,147
1,★ Driver Gloves | Racing Green,MW,0.000172,145
2,★ Moto Gloves | Boom!,WW,0.000161,135
3,★ Bloodhound Gloves | Bronzed,FT,0.004228,146
4,★ Hand Wraps | Duct Tape,FN,0.000001,7


Четвертый квантиль:

In [79]:
maxs = []
mins = []
for quality in qualities:
    max_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (vars_info_tmp['total_solds'] >= quantiles[quality][2])]['var'].max()
    min_var = vars_info_tmp[(vars_info_tmp['quality'] == quality) & (vars_info_tmp['total_solds'] >= quantiles[quality][2])]['var'].min()
    maxs += vars_info_tmp[vars_info_tmp['var'] == max_var].to_dict(orient='records')
    mins += vars_info_tmp[vars_info_tmp['var'] == min_var].to_dict(orient='records')

In [80]:
pd.DataFrame(maxs)

,market_hash_name,quality,var,total_solds
0,★ Hand Wraps | Cobalt Skulls,BS,0.011728,146
1,★ Bloodhound Gloves | Guerrilla,MW,0.000399,146
2,★ Driver Gloves | Convoy,WW,0.000482,140
3,★ Hand Wraps | Duct Tape,FT,0.006078,147
4,★ Bloodhound Gloves | Charred,FN,0.000013,5


In [81]:
pd.DataFrame(mins)

,market_hash_name,quality,var,total_solds
0,★ Specialist Gloves | Foundation,BS,0.008430,147
1,★ Driver Gloves | Racing Green,MW,0.000172,145
2,★ Moto Gloves | Boom!,WW,0.000161,135
3,★ Bloodhound Gloves | Bronzed,FT,0.004228,146
4,★ Hand Wraps | Duct Tape,FN,0.000001,7


# Проверить гипотезу: параметр `price` перчаток с параметром `case == Clutch Case`, в среднем сильнее зависит от показателя `wear_value`, чем с параметром `case == Glove Case, Operation Hydra Case`

Для проверки данной гипотезы найдем коэффициент корреляции между `price` и `wear_value` для каждой пары `market_hash_name` `quality`:

In [189]:
corrcoefs = []
for market_hash_name, quality in itertools.product(market_hash_names, qualities):
    data = df[(df['market_hash_name'] == market_hash_name) & (df['quality'] == quality)]
    if len(data) > 2:
        corrcoefs.append({'market_hash_name': market_hash_name,
                          'quality': quality, 
                          'corrcoef': np.corrcoef(data['wear_value'], data['price'])[0][1]})
corrcoefs = pd.DataFrame(corrcoefs)
corrcoefs

,market_hash_name,quality,corrcoef
0,★ Driver Gloves | Lunar Weave,BS,0.002344
1,★ Driver Gloves | Lunar Weave,MW,-0.478618
2,★ Driver Gloves | Lunar Weave,WW,-0.046205
3,★ Driver Gloves | Lunar Weave,FT,-0.881767
4,★ Driver Gloves | Lunar Weave,FN,-0.609344
...,...,...,...
206,★ Sport Gloves | Omega,FT,-0.834759
207,★ Sport Gloves | Amphibious,BS,-0.115613
208,★ Sport Gloves | Amphibious,MW,-0.500288
209,★ Sport Gloves | Amphibious,WW,-0.250141


Добавим столбец `case`

In [190]:
for index, row in corrcoefs.iterrows():
     corrcoefs.loc[index, 'case'] = df[df['market_hash_name'] == row['market_hash_name']]['case'].values[0]
corrcoefs

,market_hash_name,quality,corrcoef,case
0,★ Driver Gloves | Lunar Weave,BS,0.002344,"Glove Case, Operation Hydra Case"
1,★ Driver Gloves | Lunar Weave,MW,-0.478618,"Glove Case, Operation Hydra Case"
2,★ Driver Gloves | Lunar Weave,WW,-0.046205,"Glove Case, Operation Hydra Case"
3,★ Driver Gloves | Lunar Weave,FT,-0.881767,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Lunar Weave,FN,-0.609344,"Glove Case, Operation Hydra Case"
...,...,...,...,...
206,★ Sport Gloves | Omega,FT,-0.834759,Clutch Case
207,★ Sport Gloves | Amphibious,BS,-0.115613,Clutch Case
208,★ Sport Gloves | Amphibious,MW,-0.500288,Clutch Case
209,★ Sport Gloves | Amphibious,WW,-0.250141,Clutch Case


Исключим из выборки строки, у которых `corrcoef == NaN`, а для остальных заменим значение `corrcoef` на его модуль:

In [191]:
corrcoefs = corrcoefs[corrcoefs['corrcoef'].isnull() == False]
corrcoefs['corrcoef'] = abs(corrcoefs['corrcoef'])
corrcoefs

,market_hash_name,quality,corrcoef,case
0,★ Driver Gloves | Lunar Weave,BS,0.002344,"Glove Case, Operation Hydra Case"
1,★ Driver Gloves | Lunar Weave,MW,0.478618,"Glove Case, Operation Hydra Case"
2,★ Driver Gloves | Lunar Weave,WW,0.046205,"Glove Case, Operation Hydra Case"
3,★ Driver Gloves | Lunar Weave,FT,0.881767,"Glove Case, Operation Hydra Case"
4,★ Driver Gloves | Lunar Weave,FN,0.609344,"Glove Case, Operation Hydra Case"
...,...,...,...,...
206,★ Sport Gloves | Omega,FT,0.834759,Clutch Case
207,★ Sport Gloves | Amphibious,BS,0.115613,Clutch Case
208,★ Sport Gloves | Amphibious,MW,0.500288,Clutch Case
209,★ Sport Gloves | Amphibious,WW,0.250141,Clutch Case


Найдем среднее значение `corrcoef` для `case == Glove Case, Operation Hydra Case` и для `case == Clutch Case`:

In [192]:
corrcoefs[corrcoefs['case'] == 'Glove Case, Operation Hydra Case']['corrcoef'].mean()

0.3351601195467574

In [193]:
corrcoefs[corrcoefs['case'] == 'Clutch Case']['corrcoef'].mean()

0.3608439150807604

Проверим отношения `corrcoef` для разных значений `quality`:

In [194]:
for quality in qualities:
    print(quality, corrcoefs[(corrcoefs['case'] == 'Clutch Case') & (corrcoefs['quality'] == quality)]['corrcoef'].mean() / corrcoefs[(corrcoefs['case'] == 'Glove Case, Operation Hydra Case') & (corrcoefs['quality'] == quality)]['corrcoef'].mean())

BS 0.9429580147944352
MW 1.3031927423602565
WW 1.8215984969068204
FT 1.0004847808916226
FN 0.8527970861907818


Как видно, значения цена перчаток с `case =  Clutch Case` в среднем сильнее зависит только для качеств `MW` и `WW`

# Составить рейтинг перчаток, которые имеют сильную зависимость цены от `wear_value`, но которые можно чаще всего купить за стандартную цену. Посчитать среднюю прибыль перепродажи таких предметов за `n` дней.

"Сильной" зависимостью цены от `wear_value` будем считать перчатки, у которых модуль коэффициента корреляции больше 0.7

In [229]:
max_corrcoefs = corrcoefs[corrcoefs['corrcoef'] > 0.7]
max_corrcoefs.sort_values('corrcoef')

,market_hash_name,quality,corrcoef,case
31,★ Specialist Gloves | Crimson Web,MW,0.704727,Clutch Case
178,★ Hydra Gloves | Case Hardened,FT,0.721788,Clutch Case
143,★ Sport Gloves | Pandora's Box,FT,0.723827,"Glove Case, Operation Hydra Case"
29,★ Hand Wraps | Badlands,FT,0.724509,"Glove Case, Operation Hydra Case"
156,★ Hand Wraps | Overprint,FT,0.734666,Clutch Case
197,★ Moto Gloves | POW!,FT,0.744205,Clutch Case
78,★ Specialist Gloves | Mogul,FT,0.744333,Clutch Case
210,★ Sport Gloves | Amphibious,FT,0.748915,Clutch Case
201,★ Moto Gloves | Transport,FT,0.750718,Clutch Case
83,★ Hand Wraps | Slaughter,FT,0.751524,"Glove Case, Operation Hydra Case"


Стандартной ценой будем считать математическое ожидание +- 10%

In [230]:
for index, row in max_corrcoefs.iterrows():
    mean = df[(df['market_hash_name'] == row['market_hash_name']) & (df['quality'] == row['quality'])]['price'].mean()
    max_corrcoefs.loc[index, 'mean'] = mean
max_corrcoefs

/home/duckluckbreakout/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:844: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/duckluckbreakout/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


,market_hash_name,quality,corrcoef,case,mean
3,★ Driver Gloves | Lunar Weave,FT,0.881767,"Glove Case, Operation Hydra Case",132.233333
12,★ Specialist Gloves | Emerald Web,FT,0.901683,"Glove Case, Operation Hydra Case",352.879357
17,★ Driver Gloves | Imperial Plaid,FT,0.882125,Clutch Case,327.653692
29,★ Hand Wraps | Badlands,FT,0.724509,"Glove Case, Operation Hydra Case",156.798182
31,★ Specialist Gloves | Crimson Web,MW,0.704727,Clutch Case,616.841377
33,★ Specialist Gloves | Crimson Web,FT,0.840944,Clutch Case,304.292571
39,★ Sport Gloves | Bronze Morph,FN,0.767259,Clutch Case,917.333333
43,★ Specialist Gloves | Foundation,FT,0.872605,"Glove Case, Operation Hydra Case",242.704962
51,★ Driver Gloves | Overtake,FT,0.773277,Clutch Case,138.681397
55,★ Specialist Gloves | Fade,FT,0.911120,Clutch Case,432.448815


Для каждого элемента построим модель линейной регресии:

In [256]:
from sklearn.linear_model import LinearRegression

for index, row in max_corrcoefs.iterrows():
    x = df[(df['market_hash_name'] == row['market_hash_name']) & (df['quality'] == row['quality'])]['wear_value'].values.reshape((-1, 1))
    y = df[(df['market_hash_name'] == row['market_hash_name']) & (df['quality'] == row['quality'])]['price'].values
    model = LinearRegression().fit(x, y)
    max_corrcoefs.loc[index, 'model'] = model
max_corrcoefs

/home/duckluckbreakout/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/duckluckbreakout/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/duckluckbreakout/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:965: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

,market_hash_name,quality,corrcoef,case,mean,model
3,★ Driver Gloves | Lunar Weave,FT,0.881767,"Glove Case, Operation Hydra Case",132.233333,"LinearRegression(copy_X=True, fit_intercept=Tr..."
12,★ Specialist Gloves | Emerald Web,FT,0.901683,"Glove Case, Operation Hydra Case",352.879357,"LinearRegression(copy_X=True, fit_intercept=Tr..."
17,★ Driver Gloves | Imperial Plaid,FT,0.882125,Clutch Case,327.653692,"LinearRegression(copy_X=True, fit_intercept=Tr..."
29,★ Hand Wraps | Badlands,FT,0.724509,"Glove Case, Operation Hydra Case",156.798182,"LinearRegression(copy_X=True, fit_intercept=Tr..."
31,★ Specialist Gloves | Crimson Web,MW,0.704727,Clutch Case,616.841377,"LinearRegression(copy_X=True, fit_intercept=Tr..."
33,★ Specialist Gloves | Crimson Web,FT,0.840944,Clutch Case,304.292571,"LinearRegression(copy_X=True, fit_intercept=Tr..."
39,★ Sport Gloves | Bronze Morph,FN,0.767259,Clutch Case,917.333333,"LinearRegression(copy_X=True, fit_intercept=Tr..."
43,★ Specialist Gloves | Foundation,FT,0.872605,"Glove Case, Operation Hydra Case",242.704962,"LinearRegression(copy_X=True, fit_intercept=Tr..."
51,★ Driver Gloves | Overtake,FT,0.773277,Clutch Case,138.681397,"LinearRegression(copy_X=True, fit_intercept=Tr..."
55,★ Specialist Gloves | Fade,FT,0.911120,Clutch Case,432.448815,"LinearRegression(copy_X=True, fit_intercept=Tr..."


Найдем все предметы, которые были проданы за стандартную цену или дешевле, но могли быть проданы дороже стандартной цены:

Вначале получим все продажи предметов из списка выше:

In [285]:
max_corrcoefs_solds = []
max_std_prices = []
for index, row in max_corrcoefs.iterrows():
    solds = df[(df['market_hash_name'] == row['market_hash_name']) & (df['quality'] == row['quality'])].to_dict(orient='records')
    max_corrcoefs_solds += solds
    max_std_prices += [max_corrcoefs[(max_corrcoefs['market_hash_name'] == row['market_hash_name']) & (max_corrcoefs['quality'] == row['quality'])]['mean'].values[0]*1.1] * len(solds)
max_corrcoefs_solds = pd.DataFrame(max_corrcoefs_solds)
max_corrcoefs_solds['max_std_price'] = max_std_prices
max_corrcoefs_solds

,market_hash_name,price,wear_value,sold_at,quality,case,max_std_price
0,★ Driver Gloves | Lunar Weave,162.99,0.180964,1591249807,FT,"Glove Case, Operation Hydra Case",145.456667
1,★ Driver Gloves | Lunar Weave,127.89,0.349751,1589636335,FT,"Glove Case, Operation Hydra Case",145.456667
2,★ Driver Gloves | Lunar Weave,142.00,0.259338,1596061392,FT,"Glove Case, Operation Hydra Case",145.456667
3,★ Driver Gloves | Lunar Weave,130.00,0.242543,1595146966,FT,"Glove Case, Operation Hydra Case",145.456667
4,★ Driver Gloves | Lunar Weave,122.90,0.378014,1590124482,FT,"Glove Case, Operation Hydra Case",145.456667
...,...,...,...,...,...,...,...
4117,★ Sport Gloves | Amphibious,680.00,0.152401,1593761448,FT,Clutch Case,508.551438
4118,★ Sport Gloves | Amphibious,415.00,0.271489,1587968530,FT,Clutch Case,508.551438
4119,★ Sport Gloves | Amphibious,539.00,0.217008,1589576658,FT,Clutch Case,508.551438
4120,★ Sport Gloves | Amphibious,439.99,0.364688,1588260053,FT,Clutch Case,508.551438


После исключим из выборки продажи, цена которых выше стандартной цены:

In [286]:
max_corrcoefs_solds = max_corrcoefs_solds[max_corrcoefs_solds['price'] <= max_corrcoefs_solds['max_std_price']]

Найдем примерную реальную цену данных предметов:

In [288]:
pred_prices = []
for index, row in max_corrcoefs_solds.iterrows():
    model = max_corrcoefs[(max_corrcoefs['market_hash_name'] == row['market_hash_name']) & (max_corrcoefs['quality'] == max_corrcoefs['quality'])]['model'].values[0]
    pred_price = model.predict(np.array(row['wear_value']).reshape((-1, 1)))[0]
    pred_prices.append(pred_price)
max_corrcoefs_solds['pred_price'] = pred_prices
max_corrcoefs_solds

/home/duckluckbreakout/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,market_hash_name,price,wear_value,sold_at,quality,case,max_std_price,pred_price
1,★ Driver Gloves | Lunar Weave,127.89,0.349751,1589636335,FT,"Glove Case, Operation Hydra Case",145.456667,114.858882
2,★ Driver Gloves | Lunar Weave,142.00,0.259338,1596061392,FT,"Glove Case, Operation Hydra Case",145.456667,144.831626
3,★ Driver Gloves | Lunar Weave,130.00,0.242543,1595146966,FT,"Glove Case, Operation Hydra Case",145.456667,150.399468
4,★ Driver Gloves | Lunar Weave,122.90,0.378014,1590124482,FT,"Glove Case, Operation Hydra Case",145.456667,105.489324
5,★ Driver Gloves | Lunar Weave,139.90,0.287655,1589961125,FT,"Glove Case, Operation Hydra Case",145.456667,135.444264
...,...,...,...,...,...,...,...,...
4115,★ Sport Gloves | Amphibious,499.95,0.256267,1586066445,FT,Clutch Case,508.551438,490.660795
4116,★ Sport Gloves | Amphibious,361.74,0.281807,1568824836,FT,Clutch Case,508.551438,468.985799
4118,★ Sport Gloves | Amphibious,415.00,0.271489,1587968530,FT,Clutch Case,508.551438,477.742622
4120,★ Sport Gloves | Amphibious,439.99,0.364688,1588260053,FT,Clutch Case,508.551438,398.648250


Найдем примерную прибыль за перепродажу каждого предмета:

In [289]:
max_corrcoefs_solds['profit'] = max_corrcoefs_solds['pred_price'] - max_corrcoefs_solds['price']
max_corrcoefs_solds

/home/duckluckbreakout/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,market_hash_name,price,wear_value,sold_at,quality,case,max_std_price,pred_price,profit
1,★ Driver Gloves | Lunar Weave,127.89,0.349751,1589636335,FT,"Glove Case, Operation Hydra Case",145.456667,114.858882,-13.031118
2,★ Driver Gloves | Lunar Weave,142.00,0.259338,1596061392,FT,"Glove Case, Operation Hydra Case",145.456667,144.831626,2.831626
3,★ Driver Gloves | Lunar Weave,130.00,0.242543,1595146966,FT,"Glove Case, Operation Hydra Case",145.456667,150.399468,20.399468
4,★ Driver Gloves | Lunar Weave,122.90,0.378014,1590124482,FT,"Glove Case, Operation Hydra Case",145.456667,105.489324,-17.410676
5,★ Driver Gloves | Lunar Weave,139.90,0.287655,1589961125,FT,"Glove Case, Operation Hydra Case",145.456667,135.444264,-4.455736
...,...,...,...,...,...,...,...,...,...
4115,★ Sport Gloves | Amphibious,499.95,0.256267,1586066445,FT,Clutch Case,508.551438,490.660795,-9.289205
4116,★ Sport Gloves | Amphibious,361.74,0.281807,1568824836,FT,Clutch Case,508.551438,468.985799,107.245799
4118,★ Sport Gloves | Amphibious,415.00,0.271489,1587968530,FT,Clutch Case,508.551438,477.742622,62.742622
4120,★ Sport Gloves | Amphibious,439.99,0.364688,1588260053,FT,Clutch Case,508.551438,398.648250,-41.341750


Исключим все строки, в которых прибыль отрицательная

In [290]:
max_corrcoefs_solds = max_corrcoefs_solds[max_corrcoefs_solds['profit'] > 0]
max_corrcoefs_solds

,market_hash_name,price,wear_value,sold_at,quality,case,max_std_price,pred_price,profit
2,★ Driver Gloves | Lunar Weave,142.00,0.259338,1596061392,FT,"Glove Case, Operation Hydra Case",145.456667,144.831626,2.831626
3,★ Driver Gloves | Lunar Weave,130.00,0.242543,1595146966,FT,"Glove Case, Operation Hydra Case",145.456667,150.399468,20.399468
7,★ Driver Gloves | Lunar Weave,103.50,0.333338,1596611507,FT,"Glove Case, Operation Hydra Case",145.456667,120.299827,16.799827
10,★ Driver Gloves | Lunar Weave,110.00,0.243002,1595290155,FT,"Glove Case, Operation Hydra Case",145.456667,150.247125,40.247125
13,★ Driver Gloves | Lunar Weave,98.57,0.357004,1596632095,FT,"Glove Case, Operation Hydra Case",145.456667,112.454385,13.884385
...,...,...,...,...,...,...,...,...,...
4108,★ Sport Gloves | Amphibious,330.00,0.371779,1579151746,FT,Clutch Case,508.551438,392.630234,62.630234
4111,★ Sport Gloves | Amphibious,424.99,0.326541,1580489258,FT,Clutch Case,508.551438,431.022477,6.032477
4113,★ Sport Gloves | Amphibious,479.99,0.211575,1580590476,FT,Clutch Case,508.551438,528.589110,48.599110
4116,★ Sport Gloves | Amphibious,361.74,0.281807,1568824836,FT,Clutch Case,508.551438,468.985799,107.245799


Найдем суммарную прибыль для каждого типа предмета, а также разницу в днях между первой и последней продажами

In [305]:
profit_info = []
market_hash_names = max_corrcoefs_solds['market_hash_name'].unique()

for name, quality in itertools.product(market_hash_names, qualities):
    profit_sum = max_corrcoefs_solds[(max_corrcoefs_solds['market_hash_name'] == name) & (max_corrcoefs_solds['quality'] == quality)]['profit'].sum()
    date_difference = sorted(max_corrcoefs_solds[(max_corrcoefs_solds['market_hash_name'] == name) & (max_corrcoefs_solds['quality'] == quality)]['sold_at'].values)
    if profit_sum:
        profit_info.append({'market_hash_name': name,
                            'quality': quality,
                            'profit_sum': profit_sum,
                            'date_difference': (date_difference[-1] - date_difference[0]) / 60 / 60 / 24})
profit_info = pd.DataFrame(profit_info)
profit_info

,market_hash_name,quality,profit_sum,date_difference
0,★ Driver Gloves | Lunar Weave,FT,622.932876,85.498310
1,★ Specialist Gloves | Emerald Web,FT,1487.404032,176.279178
2,★ Driver Gloves | Imperial Plaid,FT,1329.056633,106.505856
3,★ Hand Wraps | Badlands,FT,606.829354,178.449433
4,★ Specialist Gloves | Crimson Web,MW,4669.690311,563.757512
5,★ Specialist Gloves | Crimson Web,FT,143.454926,6.351377
6,★ Sport Gloves | Bronze Morph,FN,290.236840,0.000000
7,★ Specialist Gloves | Foundation,FT,897.352912,140.366181
8,★ Driver Gloves | Overtake,FT,753.761760,88.242350
9,★ Specialist Gloves | Fade,FT,1864.964098,143.166250


In [308]:
profit_info['profit_per_day'] = profit_info['profit_sum'] / profit_info['date_difference']
profit_info.sort_values('profit_per_day', ascending=False)

,market_hash_name,quality,profit_sum,date_difference,profit_per_day
6,★ Sport Gloves | Bronze Morph,FN,290.236840,0.000000,inf
22,★ Sport Gloves | Pandora's Box,FT,9439.441997,301.827824,31.274261
18,★ Sport Gloves | Vice,FT,5982.090325,224.149549,26.687943
5,★ Specialist Gloves | Crimson Web,FT,143.454926,6.351377,22.586428
27,★ Moto Gloves | Spearmint,FT,6705.179526,369.165845,18.163055
9,★ Specialist Gloves | Fade,FT,1864.964098,143.166250,13.026562
21,★ Sport Gloves | Hedge Maze,FT,4306.983645,341.507662,12.611675
2,★ Driver Gloves | Imperial Plaid,FT,1329.056633,106.505856,12.478719
16,★ Moto Gloves | Polygon,FT,1231.893338,124.967882,9.857680
14,★ Hand Wraps | Cobalt Skulls,FT,1936.244147,213.883310,9.052806
